# Horse Racing Prediction by Deep Learning

This AI Horse racing prediction is using a 3-layer Neural Network to predict the finishing position of each horse in a race. The data used to train this model is obtained from the Hong Kong Jockey Club (HKJC). The model is built and trained by Tensorflow. Supervised learning is used in this project to classify the expected finishing position of the horses.

<H2> Part 1: Data Input and Preprocessing </H2>

In this part of the program, we will import the data obtained from the HKJC. First of all, the following features were selected based on my past horse picking experience, namely:

- position: The starting position of the horse. If the position is "1", it indicates the closest position to the hurdle and should be benficial in non-straight race courses.

- load: This is the loading of the horse in pounds. Maximum is 133.

- ON odds: This is the overnight odds of the horse provided by the HKJC.

- odds: This is the odds of the horse 15 min before the race.

- class: This is the class of the case. It is common to all horses in a race except special races.

- num horses: This is the number of horses participated in the race.


In [1]:
#Loading the data and preprocessing it.

import pandas as pd
import tensorflow as tf
tf.__version__

PATH_TRAINING_DATA = 'training_data/horse_data_train_test.csv'

dataset = pd.read_csv(PATH_TRAINING_DATA)
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0, shuffle=False)


In [2]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train) #use "fit_transform" for training data
X_test = sc.transform(X_test)       #use "transform" for testing data

y_train = tf.keras.utils.to_categorical((y_train-1), 14)
y_test = tf.keras.utils.to_categorical((y_test-1), 14)

print(X_train)

[[-0.12255789  0.80382177 -0.43396319 -0.65700651 -0.66090701 -1.24285142]
 [-0.61862553 -0.64653206 -0.75597865 -0.865751   -0.66090701 -1.24285142]
 [ 0.62154358 -1.04208311 -0.55781529 -0.6848391  -0.66090701 -1.24285142]
 ...
 [-1.362727    0.54012107 -0.79313428 -0.91306642 -0.66090701  0.56609328]
 [ 0.12547593  0.40827073 -0.58877832 -0.62917391 -0.66090701  0.56609328]
 [-1.11469317  0.27642038 -0.34107411  0.37279965 -0.66090701  0.56609328]]


<H2> Part 2: Building the Model </H2>

We will construct a basic 3-layer model: one input layer, one output layer and one hidden layer.
I have tried relu and sigmoid as the activation, and found that sigmoid seems to be a better choice for this project.


In [3]:
# Initializing the ANN
model = tf.keras.models.Sequential()

# Adding the input layer and the first layer
# We have six features now, and one bias term, so the input layer has a size of 6+1
#model.add(tf.keras.layers.Dense(units=6, activation='sigmoid'))
model.add(tf.keras.layers.Dense(units=7, activation='sigmoid'))
#model.add(tf.keras.layers.Dense(units=7, activation='relu'))

# Adding the input layer and the hidden layer
# We set the size of the hidden layer to be the mean of input and output later, i.e. 10
model.add(tf.keras.layers.Dense(units=6, activation='sigmoid')) 
#model.add(tf.keras.layers.Dense(units=10, activation='sigmoid')) 
#model.add(tf.keras.layers.Dense(units=10, activation='relu')) 

# Adding a drop out layer
model.add(tf.keras.layers.Dropout(0.2))

#model.add(tf.keras.layers.Dense(units=6, activation='sigmoid')) 
#model.add(tf.keras.layers.Dropout(0.2))

# Adding the output layer
# We have 14 outputs, so the output later has a size of 14
model.add(tf.keras.layers.Dense(units=14, activation='softmax'))

# Compiling the model
#opt = tf.keras.optimizers.Adam(learning_rate=0.03) 
opt = tf.keras.optimizers.Adam(learning_rate=0.01)
model.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics = ['accuracy'])

#model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

<H2> Part 3a: [Optional] Training and saving a new Model </H2>

In this section, we would either traing the model from scratch or just load a pre-trained model that is shipped with this Jupyter notebook.

- If the training data has been changed or the model has been renewed, go to train a new model in code cell 3a)
- If you want to save some time, just skip cell 3a) and load a pre-trained model in code cell 3b)


In [4]:
#This is code cell 3a) that trains a model from scratch and then saves it. Please note that it takes hours to train!

import tensorflow as tf
import datetime, os

%load_ext tensorboard
#%reload_ext tensorboard

logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

print("rows in X_train = " + str(X_train.shape[0]) )
print("rows in y_train = " + str(y_train.shape[0]) )

print("Training model...")
model.fit(x=X_train, 
          y=y_train, 
          batch_size = 14, 
          epochs = 5000, #10, #10000, #30000 #50000 #20000
          validation_data=(X_test, y_test), 
          callbacks=[tensorboard_callback])

# save the model for later use
print("Saving model...")
model.save('saved_model/my_model')

print("Model saved!")

rows in X_train = 425
rows in y_train = 425
Training model...
Epoch 1/5000
31/31 [==============================] - 2s 28ms/step - loss: 2.7128 - accuracy: 0.0400 - val_loss: 2.6423 - val_accuracy: 0.0374
Epoch 2/5000
31/31 [==============================] - 0s 4ms/step - loss: 2.6373 - accuracy: 0.0706 - val_loss: 2.6332 - val_accuracy: 0.0748
Epoch 3/5000
31/31 [==============================] - 0s 3ms/step - loss: 2.6308 - accuracy: 0.0682 - val_loss: 2.6237 - val_accuracy: 0.0748
Epoch 4/5000
31/31 [==============================] - 0s 5ms/step - loss: 2.6287 - accuracy: 0.0800 - val_loss: 2.6063 - val_accuracy: 0.1402
Epoch 5/5000
31/31 [==============================] - 0s 5ms/step - loss: 2.6042 - accuracy: 0.0941 - val_loss: 2.5790 - val_accuracy: 0.1308
Epoch 6/5000
31/31 [==============================] - 0s 4ms/step - loss: 2.5768 - accuracy: 0.1012 - val_loss: 2.5371 - val_accuracy: 0.1028
Epoch 7/5000
31/31 [==============================] - 0s 5ms/step - loss: 2.5245 - ac

Epoch 58/5000
31/31 [==============================] - 0s 4ms/step - loss: 2.2354 - accuracy: 0.1882 - val_loss: 2.3129 - val_accuracy: 0.1495
Epoch 59/5000
31/31 [==============================] - 0s 4ms/step - loss: 2.2227 - accuracy: 0.1953 - val_loss: 2.3143 - val_accuracy: 0.1495
Epoch 60/5000
31/31 [==============================] - 0s 5ms/step - loss: 2.2356 - accuracy: 0.1765 - val_loss: 2.3077 - val_accuracy: 0.1215
Epoch 61/5000
31/31 [==============================] - 0s 4ms/step - loss: 2.2119 - accuracy: 0.2094 - val_loss: 2.3193 - val_accuracy: 0.1215
Epoch 62/5000
31/31 [==============================] - 0s 4ms/step - loss: 2.2187 - accuracy: 0.2047 - val_loss: 2.3334 - val_accuracy: 0.1495
Epoch 63/5000
31/31 [==============================] - 0s 4ms/step - loss: 2.2239 - accuracy: 0.2000 - val_loss: 2.3151 - val_accuracy: 0.1402
Epoch 64/5000
31/31 [==============================] - 0s 5ms/step - loss: 2.2210 - accuracy: 0.2000 - val_loss: 2.3060 - val_accuracy: 0.1495

31/31 [==============================] - 0s 3ms/step - loss: 2.1827 - accuracy: 0.1976 - val_loss: 2.3787 - val_accuracy: 0.1495
Epoch 116/5000
31/31 [==============================] - 0s 4ms/step - loss: 2.1667 - accuracy: 0.2094 - val_loss: 2.3665 - val_accuracy: 0.1308
Epoch 117/5000
31/31 [==============================] - 0s 4ms/step - loss: 2.1849 - accuracy: 0.1882 - val_loss: 2.3660 - val_accuracy: 0.1121
Epoch 118/5000
31/31 [==============================] - 0s 4ms/step - loss: 2.1977 - accuracy: 0.1835 - val_loss: 2.3619 - val_accuracy: 0.1215
Epoch 119/5000
31/31 [==============================] - 0s 4ms/step - loss: 2.1912 - accuracy: 0.1906 - val_loss: 2.3728 - val_accuracy: 0.1308
Epoch 120/5000
31/31 [==============================] - 0s 4ms/step - loss: 2.2165 - accuracy: 0.1929 - val_loss: 2.3597 - val_accuracy: 0.1402
Epoch 121/5000
31/31 [==============================] - 0s 4ms/step - loss: 2.1568 - accuracy: 0.2071 - val_loss: 2.3774 - val_accuracy: 0.1215
Epoch 1

Epoch 172/5000
31/31 [==============================] - 0s 4ms/step - loss: 2.1566 - accuracy: 0.2071 - val_loss: 2.3612 - val_accuracy: 0.1495
Epoch 173/5000
31/31 [==============================] - 0s 4ms/step - loss: 2.1521 - accuracy: 0.2306 - val_loss: 2.3813 - val_accuracy: 0.1589
Epoch 174/5000
31/31 [==============================] - 0s 4ms/step - loss: 2.1573 - accuracy: 0.2094 - val_loss: 2.3608 - val_accuracy: 0.1682
Epoch 175/5000
31/31 [==============================] - 0s 4ms/step - loss: 2.1379 - accuracy: 0.2376 - val_loss: 2.3663 - val_accuracy: 0.1495
Epoch 176/5000
31/31 [==============================] - 0s 5ms/step - loss: 2.1409 - accuracy: 0.2259 - val_loss: 2.3631 - val_accuracy: 0.1589
Epoch 177/5000
31/31 [==============================] - 0s 4ms/step - loss: 2.1339 - accuracy: 0.2259 - val_loss: 2.3677 - val_accuracy: 0.1682
Epoch 178/5000
31/31 [==============================] - 0s 4ms/step - loss: 2.1140 - accuracy: 0.2306 - val_loss: 2.4061 - val_accuracy:

Epoch 229/5000
31/31 [==============================] - 0s 5ms/step - loss: 2.0941 - accuracy: 0.2376 - val_loss: 2.4418 - val_accuracy: 0.1776
Epoch 230/5000
31/31 [==============================] - 0s 4ms/step - loss: 2.1046 - accuracy: 0.2471 - val_loss: 2.4280 - val_accuracy: 0.1869
Epoch 231/5000
31/31 [==============================] - 0s 4ms/step - loss: 2.1358 - accuracy: 0.2259 - val_loss: 2.4344 - val_accuracy: 0.1869
Epoch 232/5000
31/31 [==============================] - 0s 5ms/step - loss: 2.1072 - accuracy: 0.2306 - val_loss: 2.4239 - val_accuracy: 0.2150
Epoch 233/5000
31/31 [==============================] - 0s 4ms/step - loss: 2.1165 - accuracy: 0.2282 - val_loss: 2.4453 - val_accuracy: 0.1776
Epoch 234/5000
31/31 [==============================] - 0s 5ms/step - loss: 2.1143 - accuracy: 0.2588 - val_loss: 2.4298 - val_accuracy: 0.2056
Epoch 235/5000
31/31 [==============================] - 0s 4ms/step - loss: 2.1403 - accuracy: 0.2235 - val_loss: 2.4300 - val_accuracy:

...

31/31 [==============================] - 0s 4ms/step - loss: 1.9417 - accuracy: 0.3035 - val_loss: 7.1815 - val_accuracy: 0.2336
Epoch 4724/5000
31/31 [==============================] - 0s 4ms/step - loss: 1.9264 - accuracy: 0.3059 - val_loss: 7.1494 - val_accuracy: 0.2243
Epoch 4725/5000
31/31 [==============================] - 0s 3ms/step - loss: 1.9527 - accuracy: 0.2988 - val_loss: 7.2329 - val_accuracy: 0.2523
Epoch 4726/5000
31/31 [==============================] - 0s 3ms/step - loss: 1.9109 - accuracy: 0.3082 - val_loss: 7.1351 - val_accuracy: 0.2336
Epoch 4727/5000
31/31 [==============================] - 0s 3ms/step - loss: 1.9481 - accuracy: 0.3129 - val_loss: 7.1024 - val_accuracy: 0.2430
Epoch 4728/5000
31/31 [==============================] - 0s 5ms/step - loss: 1.9057 - accuracy: 0.3059 - val_loss: 7.1659 - val_accuracy: 0.2617
Epoch 4729/5000
31/31 [==============================] - 0s 4ms/step - loss: 1.9073 - accuracy: 0.3129 - val_loss: 7.2610 - val_accuracy: 0.2336
E

31/31 [==============================] - 0s 4ms/step - loss: 1.9036 - accuracy: 0.3153 - val_loss: 7.4232 - val_accuracy: 0.2430
Epoch 4780/5000
31/31 [==============================] - 0s 4ms/step - loss: 1.9281 - accuracy: 0.3106 - val_loss: 7.4431 - val_accuracy: 0.2336
Epoch 4781/5000
31/31 [==============================] - 0s 4ms/step - loss: 1.8986 - accuracy: 0.3176 - val_loss: 7.4205 - val_accuracy: 0.2243
Epoch 4782/5000
31/31 [==============================] - 0s 4ms/step - loss: 1.8840 - accuracy: 0.3271 - val_loss: 7.3448 - val_accuracy: 0.2336
Epoch 4783/5000
31/31 [==============================] - 0s 4ms/step - loss: 1.9478 - accuracy: 0.3082 - val_loss: 7.3425 - val_accuracy: 0.2150
Epoch 4784/5000
31/31 [==============================] - 0s 4ms/step - loss: 1.9115 - accuracy: 0.3012 - val_loss: 7.4121 - val_accuracy: 0.2430
Epoch 4785/5000
31/31 [==============================] - 0s 4ms/step - loss: 1.9548 - accuracy: 0.2941 - val_loss: 7.3207 - val_accuracy: 0.2336
E

31/31 [==============================] - 0s 4ms/step - loss: 1.9285 - accuracy: 0.3153 - val_loss: 7.7132 - val_accuracy: 0.2523
Epoch 4836/5000
31/31 [==============================] - 0s 4ms/step - loss: 1.9301 - accuracy: 0.3035 - val_loss: 7.5333 - val_accuracy: 0.2523
Epoch 4837/5000
31/31 [==============================] - 0s 4ms/step - loss: 1.9558 - accuracy: 0.3059 - val_loss: 7.4802 - val_accuracy: 0.2336
Epoch 4838/5000
31/31 [==============================] - 0s 4ms/step - loss: 1.9656 - accuracy: 0.2894 - val_loss: 7.5201 - val_accuracy: 0.2336
Epoch 4839/5000
31/31 [==============================] - 0s 4ms/step - loss: 1.9247 - accuracy: 0.2941 - val_loss: 7.6054 - val_accuracy: 0.2617
Epoch 4840/5000
31/31 [==============================] - 0s 4ms/step - loss: 1.9887 - accuracy: 0.2894 - val_loss: 7.5543 - val_accuracy: 0.2617
Epoch 4841/5000
31/31 [==============================] - 0s 4ms/step - loss: 1.9847 - accuracy: 0.2776 - val_loss: 7.4527 - val_accuracy: 0.2523
E

31/31 [==============================] - 0s 4ms/step - loss: 1.9454 - accuracy: 0.2941 - val_loss: 7.5478 - val_accuracy: 0.2430
Epoch 4892/5000
31/31 [==============================] - 0s 3ms/step - loss: 1.9475 - accuracy: 0.2965 - val_loss: 7.5959 - val_accuracy: 0.2243
Epoch 4893/5000
31/31 [==============================] - 0s 4ms/step - loss: 1.9456 - accuracy: 0.3012 - val_loss: 7.5480 - val_accuracy: 0.2336
Epoch 4894/5000
31/31 [==============================] - 0s 4ms/step - loss: 1.9173 - accuracy: 0.3106 - val_loss: 7.5448 - val_accuracy: 0.2336
Epoch 4895/5000
31/31 [==============================] - 0s 4ms/step - loss: 1.9381 - accuracy: 0.3082 - val_loss: 7.5477 - val_accuracy: 0.2430
Epoch 4896/5000
31/31 [==============================] - 0s 4ms/step - loss: 1.9345 - accuracy: 0.3106 - val_loss: 7.5752 - val_accuracy: 0.2243
Epoch 4897/5000
31/31 [==============================] - 0s 4ms/step - loss: 1.9025 - accuracy: 0.3176 - val_loss: 7.4704 - val_accuracy: 0.2430
E

31/31 [==============================] - 0s 5ms/step - loss: 1.9199 - accuracy: 0.3153 - val_loss: 7.5790 - val_accuracy: 0.2243
Epoch 4948/5000
31/31 [==============================] - 0s 4ms/step - loss: 1.9372 - accuracy: 0.3035 - val_loss: 7.5491 - val_accuracy: 0.2430
Epoch 4949/5000
31/31 [==============================] - 0s 5ms/step - loss: 1.9511 - accuracy: 0.2918 - val_loss: 7.6150 - val_accuracy: 0.2336
Epoch 4950/5000
31/31 [==============================] - 0s 4ms/step - loss: 1.9354 - accuracy: 0.2965 - val_loss: 7.5750 - val_accuracy: 0.2430
Epoch 4951/5000
31/31 [==============================] - 0s 5ms/step - loss: 1.9069 - accuracy: 0.3059 - val_loss: 7.6257 - val_accuracy: 0.2617
Epoch 4952/5000
31/31 [==============================] - 0s 4ms/step - loss: 1.9392 - accuracy: 0.3035 - val_loss: 7.6329 - val_accuracy: 0.2710
Epoch 4953/5000
31/31 [==============================] - 0s 5ms/step - loss: 1.9558 - accuracy: 0.3153 - val_loss: 7.5764 - val_accuracy: 0.2617
E

<H2> Part 3b: Loading a pre-trained Model </H2>

If you have ever run cell 3a), you will load the model that you have trained. Otherwise, you will load a pre-trained model that is shipped with this Jupyter notebook.

In [5]:
#This is code cell 3b) that loads a pre-trained model. 

model = tf.keras.models.load_model('saved_model/my_model')

print("model is loaded")

model is loaded


<H2> Part 4: Evaluation </H2>

This part calculates the confusion matrix and accuracy_score of the model.

In [6]:
# Restore the weights
import numpy as np
import pandas as pd

dataset = pd.read_csv('training_data/horse_data_train_test.csv')
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)


import tensorflow as tf
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Predicting the Test set results
y_pred = model.predict( X_test )  #y_pred = new_model.predict( sc.fit_transform(X_test) ) 
y_pred = np.argmax(y_pred, axis = 1)

In [7]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, (y_pred+1))
print("cm=", cm)
print("accuracy_score=", accuracy_score(y_test, (y_pred+1)))

cm= [[ 0  2  1  0  0  0  0  0  0  0  0  0  0  0]
 [ 0 10  0  0  1  1  1  0  0  0  0  0  0  0]
 [ 0  3  4  0  1  0  0  1  0  0  0  0  0  0]
 [ 1  4  1  1  0  0  2  0  0  0  0  0  0  1]
 [ 0  3  0  0  2  0  1  0  0  0  0  0  0  0]
 [ 0  1  0  1  0  0  0  1  0  0  0  0  0  0]
 [ 0  2  0  0  1  0  2  1  0  0  0  0  0  0]
 [ 1  1  2  0  0  0  1  2  0  1  0  0  0  0]
 [ 1  0  1  0  0  0  1  0  1  0  0  1  0  0]
 [ 0  0  1  0  1  0  0  4  0  1  1  3  0  0]
 [ 0  0  2  0  5  0  0  1  0  0  0  2  1  0]
 [ 0  0  0  0  2  0  0  0  1  0  0  1  2  0]
 [ 0  1  0  0  0  0  0  1  0  0  0  1  8  0]
 [ 0  0  0  0  1  0  1  0  0  0  0  0  0  3]]
accuracy_score= 0.32710280373831774


<H2> Part 5: [Optional] Visualization of the model by Tensorboard </H2>

***Note:*** Run this section only if you have run Part3a.

By making use of the powerful visual tools given by Tensorboard, we can tune the hyper parameters of the model and visualize the results easily. If the epoch_accuracy is increasing steadily with epoch for the validation set, we could be confident that the model is on track. 

If we are not satisfy with the performaces shown in the Tensorboard, we can go back to refine the data processing, model building and hyper parameters and check on Tensorboard again iteratively.

In [8]:
print("Calling tensorboard...")

#%tensorboard --logdir logs
%tensorboard --logdir logs --port=6007 #use a different port if tensorboard fails to load!


Calling tensorboard...


Reusing TensorBoard on port 6007 (pid 13904), started 2:46:41 ago. (Use '!kill 13904' to kill it.)

<H2> Part 6: Prediction Checking </H2>

In this section, we will try to predict the results of a race on 2021/09/22, and see how accurate the model is by comparing the rediction witht the real results.
 

In [9]:
#This is the csv containing the data of a new race:

CURRENT_RACE_DATA = 'new_data/horse_data_20210922_race3.csv'
REAL_FINISHING_POSITIONS = '[ 1  6  4  2 12 10  8  3  7  5 11  9 13 14]'

In [10]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

PATH_TRAINING_DATA = 'training_data/horse_data_train_test.csv'

#training data
dataset = pd.read_csv(PATH_TRAINING_DATA)
X = dataset.iloc[:, :-1].values
Y = dataset.iloc[:, -1].values

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0, shuffle=False)

sc = StandardScaler()
X_train = sc.fit_transform(X_train)

#live data
dataset = pd.read_csv(CURRENT_RACE_DATA)
X_live = dataset.iloc[:, :-1].values

#use "transform" for live data
X_live = sc.transform(X_live)

#loading pretrained model
model = tf.keras.models.load_model('saved_model/my_model')

# Predicting the Test set results
y_pred = model.predict(X_live)  

# Predicting the finishing position
y_pred_finishing = np.argmax(y_pred, axis = 1) 
print("Expected finishing positions=", y_pred_finishing+1)

print("The real finishing positions= " + REAL_FINISHING_POSITIONS)


Expected finishing positions= [ 2  2  2  2  3  2  3  3  3  3  3  5 13 13]
The real finishing positions= [ 1  6  4  2 12 10  8  3  7  5 11  9 13 14]


For prediction on your own, you need to create a new csv file that contains the data of a new race in the folder new_data, then run the preditction.